# Data Collection

**Data Loading**

In [7]:
!pip install gdown

In [8]:
!gdown --folder "https://drive.google.com/drive/folders/14_aiMxHbjvUUJWs6XuT5of1Qd_iPWs-y" -O ../dataset/bank_data
!unzip "../dataset/bank_data/case 1.zip" -d ../dataset/bank_data

Retrieving folder contents
Processing file 1CTAUr3-03k6XaSrN-ZQjocX9dQMcvI7A case 1.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1CTAUr3-03k6XaSrN-ZQjocX9dQMcvI7A
To: /Users/altemir_1/Desktop/personalized_notifications/ai/dataset/bank_data/case 1.zip
100%|█████████████████████████████████████████| 513k/513k [00:00<00:00, 908kB/s]
Download completed
Archive:  ../dataset/bank_data/case 1.zip
   creating: ../dataset/bank_data/case 1
  inflating: ../dataset/bank_data/case 1/clients.csv  
  inflating: ../dataset/bank_data/case 1/client_10_transactions_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_10_transfers_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_11_transactions_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_11_transfers_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_12_transactions_3m.csv  
  inflating: ../dataset/bank_data/ca

OSError: [Errno 5] Input/output error

**Importing Dependencies**

In [29]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt 
import seaborn as sns

**Dependencies Set Up**

In [101]:
# Pandas set up
pd.set_option('display.max_columns', None)

# Matplotlib set up
plt.rcParams.update({
    "font.size": 14,          # default text size
    "axes.titlesize": 16,     # title size
    "axes.labelsize": 14,     # x and y labels
    "xtick.labelsize": 12,    # x tick labels
    "ytick.labelsize": 12,    # y tick labels
    "legend.fontsize": 12,    # legend
})

**Data Merging**

In [37]:
# Create dict for each type of data
transfers = []
transactions = []

# Saving all csv files in respective lists
transfers_files = glob.glob('../dataset/bank_data/case 1/*transfers*.csv')
transactions_files = glob.glob('../dataset/bank_data/case 1/*transactions*.csv')

# Looping over the files and appending to the lists
for file in transfers_files:
    df = pd.read_csv(file)
    transfers.append(df)

for file in transactions_files:
    df = pd.read_csv(file)
    transactions.append(df)

# Merging all dataframes in the lists into single dataframes    
transfers_df = pd.concat(transfers, ignore_index=True)
transactions_df = pd.concat(transactions, ignore_index=True) 
clients_df = pd.read_csv('../dataset/bank_data/case 1/clients.csv')

# Data Preparation

**Helper functions**

In [110]:
# Calculate amount spend average in 3 month for categorical feautures
def calculate_average_spend(transactions, column):
    # Group by client_id and category, then sum the amount spent
    grouped = transactions.groupby(['client_code', column])['amount'].sum().unstack(fill_value=0)
    
    # Calculate the average spent per category over 3 months
    avg_spent = grouped / 3
    
    # Reset index to turn the client_id back into a column
    avg_spent.reset_index(inplace=True)
    
    return avg_spent

# Retrieve product and status from transactions
def get_product(transactions):
    # Select relevant columns and drop duplicates to get unique client_code, product, and status combinations
    product= transactions[['client_code', 'product']].drop_duplicates()
    
    return product

def convert_currency(transactions, exchange_rate=634):
    # Convert amounts in EUR to KZT using the provided exchange rate
    transactions.loc[transactions['currency'] == 'EUR', 'amount'] *= exchange_rate
    transactions.loc[transactions['currency'] == 'EUR', 'currency'] = 'KZT'
    
    return transactions

In [111]:
# Convert currency in both dataframes
transactions_df = convert_currency(transactions_df)
transfers_df = convert_currency(transfers_df)   

# Calculate spend for each category
avg_spend_category = calculate_average_spend(transactions_df, 'category')

# Calculate average amount per transaction type
avg_transfer_type = calculate_average_spend(transfers_df, 'type')

# Calculate average amount per transfer direction
avg_transfer_direction = calculate_average_spend(transfers_df, 'direction')

# Get product and status information
product_status = get_product(transactions_df)

# Merging all dataframes into a single dataframe
merged_df = clients_df.merge(avg_spend_category, on='client_code', how='left') \
                      .merge(avg_transfer_type, on='client_code', how='left', suffixes=('', '_transfer')) \
                      .merge(avg_transfer_direction, on='client_code', how='left', suffixes=('', '_direction')) \
                      .merge(product_status, on='client_code', how='left')

merged_df.fillna(0, inplace=True)

In [140]:
merged_df

,client_code,name,status,age,city,avg_monthly_balance_KZT,АЗС,Авто,Едим дома,Играем дома,Кафе и рестораны,Кино,Косметика и Парфюмерия,Мебель,Одежда и обувь,Отели,Подарки,Продукты питания,Путешествия,Развлечения,Ремонт дома,Смотрим дома,Спорт,Такси,Ювелирные украшения,atm_withdrawal,card_in,card_out,cashback_in,cc_repayment_out,deposit_topup_out,family_in,fx_buy,fx_sell,gold_buy_out,gold_sell_in,installment_payment_out,invest_in,invest_out,loan_payment_out,p2p_out,refund_in,salary_in,stipend_in,utilities_out,in,out,product
0,1,Айгерим,Зарплатный клиент,29,Алматы,92643,51077.780000,0.00,58213.006667,51100.803333,172919.196667,58461.820000,0.000000,0.000000,0.000000,44695.690000,0.000000,215089.020000,91811.240000,2951.350000,0.000000,51811.480000,0.00,77506.703333,0.00,205982.236667,58641.356667,1.276651e+06,46091.113333,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,169336.753333,424709.753333,38157.166667,482140.576667,0.000000,164050.803333,6.250302e+05,2.240730e+06,Карта для путешествий
1,2,Данияр,Премиальный клиент,41,Астана,1577073,59478.900000,0.00,62971.683333,45220.470000,171456.690000,49816.486667,0.000000,0.000000,0.000000,53699.743333,0.000000,286542.280000,20625.133333,3527.686667,0.000000,49445.236667,0.00,71639.796667,0.00,258596.403333,68468.393333,1.217334e+06,35845.730000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,168544.470000,435235.480000,18416.316667,426027.336667,0.000000,132306.923333,5.487578e+05,2.212017e+06,Карта для путешествий
2,3,Сабина,Студент,22,Алматы,63116,35605.526667,0.00,42268.516667,53759.606667,135590.656667,40257.293333,0.000000,0.000000,0.000000,32553.786667,0.000000,150996.856667,165083.033333,8127.880000,0.000000,54425.516667,0.00,75954.596667,0.00,253208.290000,70262.920000,4.384513e+05,62415.870000,0.000000,0.000000,35945.673333,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,186608.060000,165459.110000,20268.066667,0.000000,41402.526667,131880.226667,2.302951e+05,1.175607e+06,Карта для путешествий
3,4,Тимур,Зарплатный клиент,36,Караганда,83351,51639.700000,0.00,56488.680000,33111.656667,174298.446667,45971.980000,0.000000,0.000000,0.000000,46969.100000,0.000000,215335.656667,75258.363333,4484.280000,0.000000,60958.910000,0.00,82481.033333,0.00,211585.283333,81771.823333,1.104329e+06,35703.876667,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,177172.220000,390239.300000,20492.846667,457591.840000,0.000000,151320.073333,5.955604e+05,2.034646e+06,Карта для путешествий
4,5,Камилла,Премиальный клиент,45,Алматы,1336536,77581.246667,0.00,50614.150000,77347.576667,141320.330000,43642.700000,0.000000,0.000000,0.000000,59332.150000,0.000000,236532.816667,59201.956667,8515.750000,0.000000,52019.036667,0.00,95115.953333,0.00,196831.036667,94294.413333,1.114949e+06,42196.726667,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,182681.383333,420601.590000,24976.323333,420706.336667,0.000000,151646.016667,5.821738e+05,2.066709e+06,Карта для путешествий
5,6,Аян,Стандартный клиент,34,Шымкент,131929,31412.200000,0.00,47913.590000,49662.050000,187467.026667,52626.756667,0.000000,0.000000,0.000000,63248.973333,0.000000,272877.440000,41363.366667,8040.420000,0.000000,44247.783333,0.00,91226.016667,0.00,209955.170000,93379.063333,1.206583e+06,36506.916667,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,160484.600000,443850.583333,20147.856667,459433.276667,0.000000,123396.693333,6.094671e+05,2.144270e+06,Карта для путешествий
6,7,Руслан,Премиальный клиент,48,Алматы,4040997,65181.963333,0.00,56757.786667,66721.180000,149896.856667,50095.296667,140893.986667,0.000000,0.000000,0.000000,0.000000,255698.790000,0.000000,0.000000,0.000000,36993.366667,0.00,78368.070000

# Benefits Calculation

In [ ]:
import pandas as pd
import numpy as np

def build_selection_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    From your aggregated 3-month-avg client dataframe, compute ONLY the
    features needed to select the best financial product per client.

    Output columns (per client):
      - client_code
      - avg_monthly_balance_KZT
      - total_spend
      - travel_spend
      - online_services_spend
      - premium_boosted_spend
      - top3_spend
      - fx_volume
      - non_kzt_spend_approx
      - net_cashflow
      - eligible_balance
      - deposit_activity
      - debt_like
      - invest_activity
      - gold_activity
    """

    # Ensure numeric on required inputs (create zeros if missing)
    base_num = [
        'avg_monthly_balance_KZT', 'АЗС', 'Авто', 'Едим дома', 'Играем дома',
        'Кафе и рестораны', 'Кино', 'Косметика и Парфюмерия', 'Мебель',
        'Одежда и обувь', 'Отели', 'Подарки', 'Продукты питания', 'Путешествия',
        'Развлечения', 'Ремонт дома', 'Смотрим дома', 'Спорт', 'Такси',
        'Ювелирные украшения',
        'atm_withdrawal', 'card_out', 'p2p_out',
        'salary_in', 'stipend_in', 'family_in', 'refund_in', 'cashback_in', 'invest_in',
        'loan_payment_out', 'cc_repayment_out', 'installment_payment_out',
        'deposit_topup_out', 'fx_buy', 'fx_sell', 'gold_buy_out', 'gold_sell_in',
        'in', 'out'
    ]
    for col in base_num:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            df[col] = 0.0

    # Spending categories used for card benefits
    spend_cols = [
        'АЗС','Авто','Едим дома','Играем дома','Кафе и рестораны','Кино',
        'Косметика и Парфюмерия','Мебель','Одежда и обувь','Отели','Подарки',
        'Продукты питания','Путешествия','Развлечения','Ремонт дома',
        'Смотрим дома','Спорт','Такси','Ювелирные украшения'
    ]

    # Totals / grouped spends
    total_spend = df[spend_cols].sum(axis=1).fillna(0)

    travel_spend = (df['Такси'] + df['Отели'] + df['Путешествия'] + df["Авто"]).fillna(0)

    online_services_spend = (
        df['Смотрим дома'] + df['Играем дома'] + df['Кино'] + df['Едим дома']
    ).fillna(0)

    premium_boosted_spend = (
        df['Ювелирные украшения'] + df['Косметика и Парфюмерия'] + df['Кафе и рестораны']
    ).fillna(0)

    # Top-3 categories by spend (only the sum is needed for product scoring)
    def _top3_sum(row):
        s = row[spend_cols].fillna(0)
        return float(s.sort_values(ascending=False).head(3).sum())
    top3_spend = df.apply(_top3_sum, axis=1)

    # FX / currency
    fx_volume = (df['fx_buy'] + df['fx_sell']).fillna(0)

    # Cashflow & debt signals
    income_salary = (
        df['salary_in'] + df['stipend_in'] + df['family_in'] +
        df['refund_in'] + df['cashback_in'] + df['invest_in']
    ).fillna(0)

    cash_outflow = (df['atm_withdrawal'] + df['p2p_out'] + df['card_out']).fillna(0)
    debt_like = (df['loan_payment_out'] + df['cc_repayment_out'] + df['installment_payment_out']).fillna(0)

    net_cashflow = (income_salary - (total_spend + cash_outflow + debt_like)).fillna(0)

    # Deposits & other signals
    deposit_activity = df['deposit_topup_out'].fillna(0)

    invest_activity = (df['gold_buy_out']*0)  # placeholder to keep shape if invest cols absent
    if 'invest_in' in df.columns or 'invest_out' in df.columns:
        invest_activity = (df.get('invest_in', 0).fillna(0) + df.get('invest_out', 0).fillna(0))
    gold_activity = (df['gold_buy_out'] + df['gold_sell_in']).fillna(0)

    # Assemble minimal feature frame
    out_cols = pd.DataFrame({
        'client_code': df['client_code'] if 'client_code' in df.columns else np.arange(len(df)),
        'total_spend': total_spend,
        'travel_spend': travel_spend,
        'online_services_spend': online_services_spend,
        'premium_boosted_spend': premium_boosted_spend,
        'top3_spend': top3_spend,
        'fx_volume': fx_volume,
        'net_cashflow': net_cashflow,
        'deposit_activity': deposit_activity,
        'debt_like': debt_like,
        'invest_activity': invest_activity,
        'gold_activity': gold_activity
    })

    return out_cols

In [152]:
pre_final_df = build_selection_features(merged_df)
pre_final_df

/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/2946416397.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = row[spend_cols].fillna(0)
/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/2946416397.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = row[spend_cols].fillna(0)
/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/2946416397.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=Fal

,client_code,total_spend,travel_spend,online_services_spend,premium_boosted_spend,top3_spend,fx_volume,net_cashflow,deposit_activity,debt_like,invest_activity,gold_activity
0,1,8.756381e+05,214013.633333,219587.110000,1.729192e+05,4.798195e+05,0.000000e+00,-2.385929e+06,0.000000,1.693368e+05,0.000000e+00,0.000000e+00
1,2,8.744241e+05,145964.673333,207453.876667,1.714567e+05,5.296388e+05,0.000000e+00,-2.473845e+06,0.000000,1.685445e+05,0.000000e+00,0.000000e+00
2,3,7.946233e+05,273591.416667,190710.933333,1.355907e+05,4.516705e+05,0.000000e+00,-1.678318e+06,0.000000,1.866081e+05,0.000000e+00,0.000000e+00
3,4,8.469978e+05,204708.496667,196531.226667,1.742984e+05,4.721151e+05,0.000000e+00,-2.216535e+06,0.000000,1.771722e+05,0.000000e+00,0.000000e+00
4,5,9.012237e+05,213650.060000,223623.463333,1.413203e+05,4.729691e+05,0.000000e+00,-2.328407e+06,0.000000,1.826814e+05,0.000000e+00,0.000000e+00
5,6,8.900856e+05,195838.356667,194450.180000,1.874670e+05,5.515705e+05,0.000000e+00,-2.394871e+06,0.000000,1.604846e+05,0.000000e+00,0.000000e+00
6,7,9.006073e+05,78368.070000,210567.630000,2.907908e+05,5.464896e+05,0.000000e+00,-3.018223e+06,0.000000,3.291624e+05,0.000000e+00,0.000000e+00
7,8,1.071709e+06,78509.783333,206995.746667,4.242892e+05,7.090593e+05,0.000000e+00,-3.124604e+06,0.000000,3.140596e+05,0.000000e+00,0.000000e+00
8,9,9.412963e+05,74426.613333,197628.423333,3.197684e+05,6.150759e+05,0.000000e+00,-3.116972e+06,0.000000,3.116346e+05,0.000000e+00,0.000000e+00
9,10,9.731782e+05,66312.016667,180037.280000,3.386275e+05,6.496894e+05,0.000000e+00,-3.069899e+06,0.000000,2.631481e+05,0.000000e+00,0.000000e+00
